In [1]:
from bs4 import BeautifulSoup
import requests
import time, os
import pickle

import re

## Scrape race URLs

In [5]:
# Open main results page, capture the html
URL = "https://www.crossresults.com/?n=results&map=0&region=all"
response = requests.get(URL)
print(response.status_code)
page = response.text
soup = BeautifulSoup(page,'lxml')

200


In [43]:
# race data get appeneded to each of 4 lists
race_path = []
race_name = []
race_date = []
race_region = []

# pull race path, name, date and region. 
# divs belong to 2 classes. Go through each for every month of races
for div in soup.find_all(class_="monthContent"):
    for tr in div.find_all(class_="resultsrow datarow1"):
        race_path.append(tr.find('a').get('href'))
        race_name.append(tr.find('a').text)
        race_date.append(tr.find('a').findNext().text)
        race_region.append(tr.find('a').findNext().findNext().text)
    for tr in div.find_all(class_="resultsrow datarow2"):
        race_path.append(tr.find('a').get('href'))
        race_name.append(tr.find('a').text)
        race_date.append(tr.find('a').findNext().text)
        race_region.append(tr.find('a').findNext().findNext().text)


In [44]:
# Make sure they all have the same number of items
print(len(race_path))
print(len(race_name))
print(len(race_date))
print(len(race_region))

9489
9489
9489
9489


In [51]:
# We only want races in the US
US_regions = ['California', 'Mid Atlantic', 'Mountain West', 
              'New England', 'New York/Ontario', 'North Central', 
              'Pacific Northwest', 'South Central', 'Southeast']

# These IDs are for US races
US_ids = []

# Find if region is in the US, save ID if so
for idx, region in enumerate(race_region):
    if region in US_regions:
        US_ids.append(idx)

# Update lists to only include US races
race_path = [race_path[i] for i in US_ids]
race_name = [race_name[i] for i in US_ids]
race_date = [race_date[i] for i in US_ids]
race_region = [race_region[i] for i in US_ids]

In [55]:
# Put the data into one object
races = [race_path, race_name, race_date, race_region]

In [56]:
# Pickle the race data
with open('races.pickle', 'wb') as to_write:
    pickle.dump(races, to_write)

In [57]:
race_path[0]

'/race/10159'

## Scrape data from each race

In [ ]:
# Read in the race data
with open('races.pickle','rb') as read_file:
    races = pickle.load(read_file)

In [3]:
# Open each race page, capture the html
path = '/race/10159'
URL = 'https://www.crossresults.com' + path
response = requests.get(URL)
print(response.status_code)
page = response.text
soup = BeautifulSoup(page,'lxml')

200


In [ ]:
"
# Things to collect
#Name, Date, Location
#Latitude and Longitude
#Beers, Moisture, Accel, Tech, Elevation
#weather, temperature, wind
results
"

In [136]:
# Race info
main = soup.find("div", {"id": "resultstitle"}).text.split(' • ')

# Race Name
main[0]

# Race date
main[1].split()

# Race location
main[2].split('\r')[0].strip()

'Green Cove Springs, FL'

In [138]:
# Race rating

# Beers
soup.find("div", {"class": "beerrating rating"}).text.split()[0]

# Moisture
soup.find("div", {"class": "moisturerating rating"}).text.split()[0]

# Accel
soup.find("div", {"class": "accelrating rating"}).text.split()[0]

# Tech
soup.find("div", {"class": "techrating rating"}).text.split()[0]

# Elevation
soup.find("div", {"class": "elevationrating rating"}).text.split()[0]


'2.4'

In [41]:
# Conditions
conditions = soup.find("div", {"id": "resultstitle"}).text.strip().split('\n')[-1].strip()

# Weather
conditions.split(',')[0]

# Temperature
conditions.split(',')[1].strip().split()[0]

# Wind
conditions.split(',')[2].strip().split()[1]

'4'

In [40]:
# extract script tag to get lat and lon
script = soup.find('article', {'id': 'content'}).find('script', {'type': 'text/javascript'})

# pull out the lat and lon
pattern = re.compile('GetMap\(\"(.*?)"')
re.findall(pattern, script.string)[0]

'29.9884166717529:-81.6820068359375'